<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Chapter 3 Exercise solutions

In [1]:
from importlib.metadata import version

import torch
print("torch version:", version("torch"))

torch version: 2.4.0


# Exercise 3.1

In [2]:
import torch

# 创建一个6x3的输入张量，每一行代表一个单词的向量表示
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)  - 第1个单词向量
   [0.55, 0.87, 0.66], # journey  (x^2)  - 第2个单词向量  
   [0.57, 0.85, 0.64], # starts   (x^3)  - 第3个单词向量
   [0.22, 0.58, 0.33], # with     (x^4)  - 第4个单词向量
   [0.77, 0.25, 0.10], # one      (x^5)  - 第5个单词向量
   [0.05, 0.80, 0.55]] # step     (x^6)  - 第6个单词向量
)

# 定义输入维度(d_in=3)和输出维度(d_out=2)
d_in, d_out = 3, 2

In [3]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        # 存储输出维度
        self.d_out = d_out
        # 初始化查询(Query)权重矩阵
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        # 初始化键(Key)权重矩阵 
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        # 初始化值(Value)权重矩阵
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        # 计算键向量
        keys = x @ self.W_key
        # 计算查询向量
        queries = x @ self.W_query
        # 计算值向量
        values = x @ self.W_value
        
        # 计算注意力分数 (Q·K^T)
        attn_scores = queries @ keys.T # omega
        # 计算注意力权重,使用scaled dot-product attention
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        # 计算最终的上下文向量
        context_vec = attn_weights @ values
        return context_vec

# 设置随机种子以保证结果可复现
torch.manual_seed(123)
# 实例化自注意力模型
sa_v1 = SelfAttention_v1(d_in, d_out)

In [4]:
# 自注意力机制的第二个版本实现
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        # 存储输出维度
        self.d_out = d_out
        # 使用nn.Linear层初始化查询(Query)权重矩阵,不使用偏置项
        self.W_query = nn.Linear(d_in, d_out, bias=False)
        # 使用nn.Linear层初始化键(Key)权重矩阵,不使用偏置项
        self.W_key   = nn.Linear(d_in, d_out, bias=False)
        # 使用nn.Linear层初始化值(Value)权重矩阵,不使用偏置项
        self.W_value = nn.Linear(d_in, d_out, bias=False)

    def forward(self, x):
        # 计算键向量
        keys = self.W_key(x)
        # 计算查询向量
        queries = self.W_query(x)
        # 计算值向量
        values = self.W_value(x)
        
        # 计算注意力分数 (Q·K^T)
        attn_scores = queries @ keys.T
        # 计算注意力权重,使用scaled dot-product attention
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)

        # 计算最终的上下文向量
        context_vec = attn_weights @ values
        return context_vec

# 设置随机种子以保证结果可复现
torch.manual_seed(123)
# 实例化自注意力模型
sa_v2 = SelfAttention_v2(d_in, d_out)

In [5]:
# 将v2版本的权重矩阵转置后赋值给v1版本
# 因为v1和v2的实现方式略有不同,需要转置权重矩阵以保持一致性
sa_v1.W_query = torch.nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_key = torch.nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_value = torch.nn.Parameter(sa_v2.W_value.weight.T)

In [6]:
# 使用v1版本的自注意力模型处理输入数据
sa_v1(inputs)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

In [7]:
# 使用v2版本的自注意力模型处理输入数据
sa_v2(inputs)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

# Exercise 3.2

 If we want to have an output dimension of 2, as earlier in single-head attention, we can have to change the projection dimension `d_out` to 1:
 如果我们想要得到维度为2的输出(就像之前的单头注意力一样),我们需要将投影维度 `d_out` 改为1:

```python
torch.manual_seed(123)

d_out = 1
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)
```

```
tensor([[[-9.1476e-02,  3.4164e-02],
         [-2.6796e-01, -1.3427e-03],
         [-4.8421e-01, -4.8909e-02],
         [-6.4808e-01, -1.0625e-01],
         [-8.8380e-01, -1.7140e-01],
         [-1.4744e+00, -3.4327e-01]],

        [[-9.1476e-02,  3.4164e-02],
         [-2.6796e-01, -1.3427e-03],
         [-4.8421e-01, -4.8909e-02],
         [-6.4808e-01, -1.0625e-01],
         [-8.8380e-01, -1.7140e-01],
         [-1.4744e+00, -3.4327e-01]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
```

# Exercise 3.3

```python
context_length = 1024
d_in, d_out = 768, 768
num_heads = 12

mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads)
```

Optionally, the number of parameters is as follows:

```python
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(mha)
```

```
2360064  # (2.36 M)
```

The GPT-2 model has 117M parameters in total, but as we can see, most of its parameters are not in the multi-head attention module itself.